## Prerequisites

### SyftUI
Make sure you have SyftUI running with your email logged in.
1. It could be downloaded from [SyftUI](https://github.com/OpenMined/SyftUI/releases)


### Install dependencies for this demo

### Python Environment with Jupyter

Please make sure to have a virtual python environment with `jupyter` installed to run this notebook, e.g. run the below commands in a terminal
```bash
uv venv
source .venv/bin/activate
uv pip install jupyterlab
```

Configure the notebook to run with the Python kernel from the installed environment, or run `uv run jupyterlab` 

In [ ]:
!uv pip install syftbox-enclave syft-rds==0.1.1-dev.2

In [1]:
from syft_rds import init_session
from syftbox_enclave.client import connect

## Explore Datasets

In [13]:
DO1 = "<DO1-EMAIL>"
DO2 = "<DO1-EMAIL>"
DS  = "<DS-EMAIL>"

do_client_1 = init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = init_session(host=DO2)
print("Logged into: ", do_client_2.host)


Logged into:  do-1-oc@openmined.org
Logged into:  do-2-oc@openmined.org


In [4]:
DATASET_NAME = "Organic Crop Stock Data"

dataset1 = do_client_1.dataset.get(name=DATASET_NAME)
dataset1.describe()

dataset2 = do_client_2.dataset.get(name=DATASET_NAME)
dataset2.describe()

uid,7c188815-4306-4181-ab93-252ab03e6da0
created_at,2025-06-12 10:13:43
updated_at,2025-06-12 10:13:43
name,Organic Crop Stock Data
readme_path,/Users/rasswanths/SyftBox/datasites/do-1-oc@openmined.org/public/datasets/Organic Crop Stock Data/README.md
mock_path,/Users/rasswanths/SyftBox/datasites/do-1-oc@openmined.org/public/datasets/Organic Crop Stock Data


uid,b2360c9f-e928-447d-869f-4d798ccb775a
created_at,2025-06-12 10:08:22
updated_at,2025-06-12 10:08:22
name,Organic Crop Stock Data
readme_path,/Users/rasswanths/SyftBox/datasites/do-2-oc@openmined.org/public/datasets/Organic Crop Stock Data/README.md
mock_path,/Users/rasswanths/SyftBox/datasites/do-2-oc@openmined.org/public/datasets/Organic Crop Stock Data


## Prototype Code

In [6]:
from pathlib import Path

code_path = Path(".") / "code"
code_path.mkdir(exist_ok=True)

code_file = code_path / "entrypoint.py"

In [7]:
%%writefile {code_file}

import os
from pathlib import Path
from sys import exit

import pandas as pd

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

dataset_paths = [ Path(dataset_path) for dataset_path in DATA_DIR.split(",")]
total_carrots = 0
total_tomatoes = 0

for dataset_path in dataset_paths:
    if not dataset_path.exists():
        print("Warning: Dataset path does not exist:", dataset_path)
        exit(1)
    df = pd.read_csv(dataset_path / "crop_stock_data.csv")
    total_carrots += df[df["Product name"] == "Carrots"]["Quantity"].sum()
    total_tomatoes += df[df["Product name"] == "Tomatoes"]["Quantity"].sum()

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write(f"Total Carrots: {total_carrots}\n")
    f.write(f"Total Tomatoes: {total_tomatoes}\n")

Writing code/entrypoint.py


### DO Job

In [8]:
# Job submission
datasites = [do_client_1,do_client_2]

for client in datasites:
    job = client.jobs.submit(
            name="Organic Crop Test",
            description="Farming Coop Avg Experiment",
            user_code_path="./code",
            dataset_name=DATASET_NAME,
            tags=["enclave", "syft"],
            entrypoint="entrypoint.py",
            enclave = "enclave-organic-coop@openmined.org"
        )
    print(job)

Job
  uid: 365e5778-674b-491a-9c76-fdabe2964dff
  created_by: rasswanth@openmined.org
  created_at: 2025-06-12T10:18:58.747926Z
  updated_at: 2025-06-12T10:18:58.747928Z
  client_id: 26d95ace-7f20-4a63-91d7-11cfff6e1d87
  name: Organic Crop Test
  description: Farming Coop Avg Experiment
  user_code_id: 3ac41b51-30c4-4355-9c0f-b71b9f656f07
  tags: ['enclave', 'syft']
  user_metadata: {}
  status: pending_code_review
  error: no_error
  error_message: None
  output_url: syft://do-1-oc@openmined.org/app_data/RDS/user_files/rasswanth@openmined.org/Job/365e5778-674b-491a-9c76-fdabe2964dff
  dataset_name: Organic Crop Stock Data
  enclave: enclave-organic-coop@openmined.org
Job
  uid: d8b3f939-b28e-4a12-b274-3fe5bd257069
  created_by: rasswanth@openmined.org
  created_at: 2025-06-12T10:18:59.393859Z
  updated_at: 2025-06-12T10:18:59.393861Z
  client_id: 1f32e195-5e2a-4efd-9173-1505a9ca4eeb
  name: Organic Crop Test
  description: Farming Coop Avg Experiment
  user_code_id: dde2fa2b-5e95-4a7

### Enclave Client

In [11]:
enclave_client = connect("enclave-organic-coop@openmined.org")

In [12]:
proj_res = enclave_client.create_project(
                   project_name = "Organic Test Report",
                   datasets = [dataset1, dataset2] ,
                   output_owners = [DO1, DO2, DS ],
                   code_path = "./code",
                   entrypoint = "entrypoint.py"
            )
            

2025-06-12 15:54:03.312 | INFO     | syftbox_enclave.client:create_project:76 - Project Organic Test Report created in enclave app path /Users/rasswanths/SyftBox/datasites/enclave-organic-coop@openmined.org/app_data/enclave.


In [14]:
proj_res.output()

2025-06-12 15:57:27.552 | INFO     | syftbox_enclave.client:output:97 - Output available for project Organic Test Report at /Users/rasswanths/SyftBox/datasites/enclave-organic-coop@openmined.org/app_data/enclave/jobs/outputs/Organic Test Report.
